In [1]:
#PRECISO PEGAR OS DADOS DA TABELA PRINCIPAL, ESSES DADOS AI SAO DO MODAL.
#DPS DE FAZER ISSO, VOU COLOCAR OS DADOS DA TAB PRINCIAPL NO BANCO E DPS DO MODAL
# o banco vai ter 2 tabelas, uma com os dados principais e outra com os dados do mal referente a esse item principal

In [1]:
from selenium import webdriver
from core import MySQLcompatible
import re
import utils
import json
import time

In [2]:
tabelas = {}

tabelas['campeonato'] = (
"CREATE TABLE IF NOT EXISTS `campeonato` ("
" `idjogo` INT NOT NULL,"
" `nome_campeonato` varchar(100) NOT NULL,"
" `partida` varchar(150) NOT NULL,"
" `data_hora` varchar(100) NOT NULL,"    
" PRIMARY KEY (`idjogo`)"
") ENGINE=MyISAM")

tabelas['jogos'] = (
"CREATE TABLE IF NOT EXISTS `jogos` ("
" `id_jogo` INT NOT NULL,"
" `dados` VARCHAR(255) NOT NULL,"
" `valor` FLOAT(4,2),"
" FOREIGN KEY (`id_jogo`)"
" REFERENCES campeonato(`idjogo`),"
" PRIMARY KEY (id_jogo,dados)"
") ENGINE=MyISAM")

In [3]:
options = webdriver.ChromeOptions()
options.add_argument('headless')
browser = webdriver.Chrome(options=options)
browser.get("https://bets93.net/")
browser.find_element_by_xpath("//div[@class='lateral']/div/ul").click() #clica no botao "todos"

In [4]:
pattern_campeonato = re.compile(r"c_visivel")
pattern_jogo = re.compile(r"j_visivel_")

In [5]:
tabela_jogos = browser.find_element_by_class_name("jogos")
jogos = tabela_jogos.find_elements_by_tag_name("div")

with MySQLcompatible('daniel','123456789',) as database:
    db = database[0]
    cursor = database[1]
    utils.create_database(cursor, "bets93")
    utils.connect_database(cursor, "bets93")
    utils.create_table(db, cursor, tabelas)

    for jogo in jogos:
        is_camp = pattern_campeonato.match(jogo.get_attribute("id"))
        is_jogo = pattern_jogo.match(jogo.get_attribute("id"))
        if is_camp:
            campeonato = jogo.find_element_by_class_name("camp").text
        elif is_jogo:
            id_jogo = jogo.get_attribute("id").split('_')[-1]
            id_botao = "jogo_"+id_jogo+"_outros"

            while True:
                try:
                    jogo.find_element_by_id(id_botao).click()
                    break
                except Exception as err:                
                    browser.find_element_by_class_name("btn.btn-danger").click()
            time.sleep(2)                
            nome_jogo, data_hora = browser.find_element_by_xpath("//div[span/@class='style3']").text.split('\n') 

            dados_campeonato = {
            'idjogo': int(id_jogo),
            'nome_campeonato': campeonato,
            'partida': nome_jogo,
            'data_hora': data_hora,
            }
            utils.insert_into_campeonato(db, cursor, dados_campeonato)
            modal = browser.find_element_by_id("modal")
            divs_data = modal.find_elements_by_class_name("container.container-fluid")

            data = {}
            for i in range(len(divs_data)):
                data[i] = {}
                data[i]['ops'] = []
                data[i]['values'] = []
                data_split = divs_data[i].text.split('\n')

                for j in range(len(data_split)):
                    if j == 0:
                        camp_title = data_split[j]
                    elif j%2==1:
                        data[i]['ops'].append(camp_title + " " +data_split[j])
                    else:
                        data[i]['values'].append(float(data_split[j]))


            browser.find_element_by_class_name("btn.btn-danger").click()
            for key in data.values():
                for ii in range(len(key['ops'])):
                    utils.insert_into_jogos(db, cursor,{
                        'idjogo': int(id_jogo),
                        'dados': key['ops'][ii],
                        'valor': key['values'][ii],
                    })


Successfully Connected with bets93!

Creating table campeonato 
Creating table jogos 


In [ ]:
# tabela_jogos = browser.find_element_by_class_name("jogos")
# jogos = tabela_jogos.find_elements_by_tag_name("div")

# with MySQLcompatible('daniel','123456789',) as database:
#     db = database[0]
#     cursor = database[1]
#     utils.create_database(cursor, "bets93")
#     utils.connect_database(cursor, "bets93")
#     utils.create_table(cursor, tabelas, "campeonato")
    
#     for jogo in jogos:
#         is_camp = pattern_campeonato.match(jogo.get_attribute("id"))
#         is_jogo = pattern_jogo.match(jogo.get_attribute("id"))
#         if is_camp:
#             campeonato = jogo.find_element_by_class_name("camp").text
#         elif is_jogo:

#             id_jogo = jogo.get_attribute("id").split('_')[-1]
#             id_botao = "jogo_"+id_jogo+"_outros"

#             while True:
#                 try:
#                     jogo.find_element_by_id(id_botao).click()
#                     break
#                 except Exception as err:                
#                     browser.find_element_by_class_name("btn.btn-danger").click()
#             time.sleep(2)                
#             nome_jogo, data_hora = browser.find_element_by_xpath("//div[span/@class='style3']").text.split('\n') 

#             dados_campeonato = {
#             'idjogo': int(id_jogo),
#             'nome_campeonato': campeonato,
#             'partida': nome_jogo,
#             'data_hora': data_hora,
#             }
#             utils.insert_into_campeonato(db,cursor,dados_campeonato)
#         else:
#             continue